In [17]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_df = pd.read_csv(file_to_load)

In [18]:
# Player Count
# Display total number of players
playerlist = purchase_df["SN"].unique()
total_player_dict = {"Total Players": [len(playerlist)]}
total_player = pd.DataFrame(total_player_dict)
total_player

,Total Players
0,576
